In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
 
# The GPU id to use, usually either "0" or "1";
os.environ["CUDA_VISIBLE_DEVICES"]="0";

# TensorFlow and tf.keras
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

1.15.0


Using TensorFlow backend.


In [2]:
batch_size = 128
num_classes = 10
epochs = 12

# the data, split between train and test sets
fashion_mnist = keras.datasets.fashion_mnist
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
# input image dimensions
img_rows, img_cols = 28, 28

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

In [3]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [4]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [5]:
model = Sequential()
model.add(Conv2D(16, kernel_size=(4, 4),
                 strides=2,
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(32, (4, 4), strides=2, activation='relu'))
model.add(Flatten())
model.add(Dense(100, activation='sigmoid'))
model.add(Dense(100, activation='sigmoid'))
model.add(Dense(num_classes, activation='softmax'))

W0118 15:46:49.726732 140483458029376 module_wrapper.py:139] From /usr/lib/python3/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0118 15:46:49.730771 140483458029376 module_wrapper.py:139] From /usr/lib/python3/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0118 15:46:49.735303 140483458029376 module_wrapper.py:139] From /usr/lib/python3/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [6]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

W0118 15:46:50.586909 140483458029376 module_wrapper.py:139] From /usr/lib/python3/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0118 15:46:50.594146 140483458029376 module_wrapper.py:139] From /usr/lib/python3/dist-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



In [7]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

W0118 15:46:51.677073 140483458029376 deprecation.py:323] From /home/jiameng/packages/Bernsp/lib/python3.6/site-packages/tensorflow_core/python/ops/math_grad.py:1424: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0118 15:46:51.775836 140483458029376 module_wrapper.py:139] From /usr/lib/python3/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W0118 15:46:51.780346 140483458029376 module_wrapper.py:139] From /usr/lib/python3/dist-packages/keras/backend/tensorflow_backend.py:973: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.

W0118 15:46:51.900912 140483458029376 module_wrapper.py:139] From /usr/lib/python3/dist-packages/keras/backend/tensorflow_backend.py:2741: The name tf.Session is deprecated. Please use tf.compat.v1.Session

Train on 60000 samples, validate on 10000 samples
Epoch 1/12


W0118 15:46:52.264592 140483458029376 module_wrapper.py:139] From /usr/lib/python3/dist-packages/keras/backend/tensorflow_backend.py:206: The name tf.variables_initializer is deprecated. Please use tf.compat.v1.variables_initializer instead.



60000/60000 [==============================] - 5s 84us/step - loss: 0.9101 - acc: 0.6841 - val_loss: 0.6249 - val_acc: 0.7644
Epoch 2/12
60000/60000 [==============================] - 3s 51us/step - loss: 0.5133 - acc: 0.8111 - val_loss: 0.4910 - val_acc: 0.8174
Epoch 3/12
60000/60000 [==============================] - 3s 51us/step - loss: 0.4314 - acc: 0.8412 - val_loss: 0.4796 - val_acc: 0.8194
Epoch 4/12
60000/60000 [==============================] - 3s 50us/step - loss: 0.3862 - acc: 0.8588 - val_loss: 0.3994 - val_acc: 0.8546
Epoch 5/12
60000/60000 [==============================] - 3s 50us/step - loss: 0.3566 - acc: 0.8688 - val_loss: 0.3761 - val_acc: 0.8609
Epoch 6/12
60000/60000 [==============================] - 3s 50us/step - loss: 0.3346 - acc: 0.8758 - val_loss: 0.3651 - val_acc: 0.8646
Epoch 7/12
60000/60000 [==============================] - 3s 51us/step - loss: 0.3155 - acc: 0.8827 - val_loss: 0.3478 - val_acc: 0.8743
Epoch 8/12
60000/60000 [============================

In [8]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test accuracy:', test_acc)

10000/10000 [==============================] - 0s 49us/step
Test accuracy: 0.8827


## Make predictions

In [9]:
predictions = model.predict(x_test)

In [10]:
predictions[0]

array([1.4906182e-05, 2.4832809e-06, 8.7828612e-06, 1.7055820e-04,
       4.1414067e-05, 1.1613341e-03, 1.8276971e-05, 2.8220655e-02,
       3.5572084e-05, 9.7032601e-01], dtype=float32)

In [11]:
np.argmax(predictions[0])

9

In [12]:
y_test[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.], dtype=float32)

In [13]:
# serialize model to JSON
model_json = model.to_json()
with open("/home/jiameng/packages/ReachNN/ReachNN-CNN/model/model_CNNA.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("/home/jiameng/packages/ReachNN/ReachNN-CNN/model/model_CNNA.h5")
print("Saved model to disk")

Saved model to disk


In [14]:
weights = []
for layer in model.layers:
    weights.append(layer.get_weights()) # list of numpy arrays

In [16]:
model.layers[0].get_weights()[0].shape

(4, 4, 1, 16)